# Collect Demonstration from Keyboard

Collect demonstration data for the given environment.
The task is to pick a mug and place it on the plate. The environment recognizes the success if the mug is on the plate, gthe ripper opened, and the end-effector positioned above the mug.

<img src="./media/teleop.gif" width="480" height="360">

Use WASD for the xy plane, RF for the z-axis, QE for tilt, and ARROWs for the rest of rthe otations. 

SPACEBAR will change your gripper's state, and Z key will reset your environment with discarding the current episode data.

For overlayed images, 
- Top Right: Agent View 
- Bottom Right: Egocentric View
- Top Left: Left Side View
- Bottom Left: Top View

In [2]:
import sys
import random
import numpy as np
import os
from PIL import Image
from mujoco_env.ffw_env import SimpleEnv
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

/home/heewonlee/Documents/lerobot-mujoco-tutorial/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# If you want to randomize the object positions, set this to None
# If you fix the seed, the object positions will be the same every time
SEED = 0 
# SEED = None <- Uncomment this line to randomize the object positions

REPO_NAME = 'ffw_pnp'
NUM_DEMO = 1 # Number of demonstrations to collect
ROOT = "./demo_data" # The root directory to save the demonstrations

In [4]:
TASK_NAME = 'Put mug cup on the plate' 
xml_path = './asset/example_scene_ffw.xml'
# Define the environment
PnPEnv = SimpleEnv(xml_path, seed = SEED, state_type = 'joint_angle')


-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[39] n_qvel:[37] n_qacc:[37] n_ctrl:[25]
 integrator:[IMPLICITFAST]

n_body:[34]
 [0/34] [world] mass:[0.00]kg
 [1/34] [front_object_table] mass:[1.00]kg
 [2/34] [camera] mass:[0.00]kg
 [3/34] [camera2] mass:[0.00]kg
 [4/34] [camera3] mass:[0.00]kg
 [5/34] [arm_base_link] mass:[8.72]kg
 [6/34] [head_link1] mass:[0.14]kg
 [7/34] [head_link2] mass:[0.15]kg
 [8/34] [arm_l_link1] mass:[1.93]kg
 [9/34] [arm_l_link2] mass:[2.12]kg
 [10/34] [arm_l_link3] mass:[1.41]kg
 [11/34] [arm_l_link4] mass:[1.51]kg
 [12/34] [arm_l_link5] mass:[1.34]kg
 [13/34] [arm_l_link6] mass:[0.64]kg
 [14/34] [arm_l_link7] mass:[0.38]kg
 [15/34] [gripper_l_rh_p12_rn_r1] mass:[0.07]kg
 [16/34] [gripper_l_rh_p12_rn_r2] mass:[0.02]kg
 [17/34] [gripper_l_rh_p12_rn_l1] mass:[0.07]kg
 [18/34] [gripper_l_rh_p12_rn_l2] mass:[0.02]kg
 [19/34] [arm_r_link1] mass:[1.93]kg
 [20/34] [arm_r_link2] mass:[2.12]

## Define Dataset Fatures and Create your dataset!
The dataset is contained as follows:
```
fps = 20,
features={
    "observation.image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channels"],
    },
    "observation.wrist_image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channel"],
    },
    "observation.state": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["state"], # x, y, z, roll, pitch, yaw
    },
    "action": {
        "dtype": "float32",
        "shape": (7,),
        "names": ["action"], # 6 joint angles and 1 gripper
    },
    "obj_init": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["obj_init"], # just the initial position of the object. Not used in training.
    },
},
```


This will make the dataset on './demo_data' folder, which will look like this,

```
.
├── data
│   ├── chunk-000
│   │   ├── episode_000000.parquet
│   │   └── ...
├── meta
│   ├── episodes.jsonl
│   ├── info.json
│   ├── stats.json
│   └── tasks.jsonl
└── 
```


In [5]:
create_new = True
if os.path.exists(ROOT):
    print(f"Directory {ROOT} already exists.")
    ans = input("Do you want to delete it? (y/n) ")
    if ans == 'y':
        import shutil
        shutil.rmtree(ROOT)
    else:
        create_new = False


if create_new:
    dataset = LeRobotDataset.create(
                repo_id=REPO_NAME,
                root = ROOT, 
                robot_type="ffw",
                fps=20, # 20 frames per second
                features={
                    "observation.image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channels"],
                    },
                    "observation.wrist_image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channel"],
                    },
                    "observation.state": {
                        "dtype": "float32",
                        "shape": (6,),
                        "names": ["state"], # x, y, z, roll, pitch, yaw
                    },
                    "action": {
                        "dtype": "float32",
                        "shape": (7,),
                        "names": ["action"], # 6 joint angles and 1 gripper
                    },
                    "obj_init": {
                        "dtype": "float32",
                        "shape": (6,),
                        "names": ["obj_init"], # just the initial position of the object. Not used in training.
                    },
                },
                image_writer_threads=10,
                image_writer_processes=5,
        )
else:
    print("Load from previous dataset")
    dataset = LeRobotDataset(REPO_NAME, root=ROOT)

## Keyboard Control
You can teleop your robot with keyboard and collect dataset
```
---------     -----------------------
   w       ->        backward
s  a  d        left   forward   right
---------      -----------------------
In x, y plane

---------
R: Moving Up
F: Moving Down
---------
In z axis

---------
Q: Tilt left
E: Tilt right
UP: Look Upward
Down: Look Donward
Right: Turn right
Left: Turn left
---------
For rotation

---------
SPACEBAR: Toggle Gripper
--------

---------
z: reset
--------
```
Reseting your environment will remove the cache data of the current demonstration and restart collection.

### Now let's teleop our robot and collect data!

**To receive the success signal, you have to release the gripper and move upwards above the mug!**

In [ ]:
action = np.zeros(7)
episode_id = 0
record_flag = False # Start recording when the robot starts moving
while PnPEnv.env.is_viewer_alive() and episode_id < NUM_DEMO:
    PnPEnv.step_env()
    if PnPEnv.env.loop_every(HZ=20):
        # check if the episode is done
        done = PnPEnv.check_success()
        if done: 
            # Save the episode data and reset the environment
            dataset.save_episode()
            PnPEnv.reset(seed = SEED)
            episode_id += 1
        # Teleoperate the robot and get delta end-effector pose with gripper
        action, reset  = PnPEnv.teleop_robot()
        if not record_flag and sum(action) != 0:
            record_flag = True
            print("Start recording")
        if reset:
            # Reset the environment and clear the episode buffer
            # This can be done by pressing 'z' key
            PnPEnv.reset(seed=SEED)
            # PnPEnv.reset()
            dataset.clear_episode_buffer()
            record_flag = False
        # Step the environment
        # Get the end-effector pose and images
        ee_pose = PnPEnv.get_ee_pose()
        agent_image,wrist_image = PnPEnv.grab_image()
        # # resize to 256x256
        agent_image = Image.fromarray(agent_image)
        wrist_image = Image.fromarray(wrist_image)
        agent_image = agent_image.resize((256, 256))
        wrist_image = wrist_image.resize((256, 256))
        agent_image = np.array(agent_image)
        wrist_image = np.array(wrist_image)
        joint_q = PnPEnv.step(action)
        if record_flag:
            # Add the frame to the dataset
            dataset.add_frame( {
                    "observation.image": agent_image,
                    "observation.wrist_image": wrist_image,
                    "observation.state": ee_pose, 
                    "action": joint_q,
                    "obj_init": PnPEnv.obj_init_pose,
                    # "task": TASK_NAME,
                }, task = TASK_NAME
            )
        PnPEnv.render(teleop=True)

ValueError: 'egocentric' is not in list

: 

In [ ]:
PnPEnv.env.close_viewer()

In [ ]:
# Clean up the images folder
import shutil
shutil.rmtree(dataset.root / 'images')